<a href="https://colab.research.google.com/github/LukeZaneh/RedeNeuralSimples/blob/main/ann_03_keras_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Curso: Redes Neurais e Deep Learning

Prof. Denilson Alves Pereira 
https://sites.google.com/ufla.br/denilsonpereira/ 
Departamento de Ciência da Computação - 
Instituto de Ciências Exatas e Tecnológicas - 
Universidade Federal de Lavras

## Introdução ao Framework Keras

Objetivo: apresentar um exemplo de aplicação da rede neural usando a API Keras. Keras é uma biblioteca de código aberto para criação de redes neurais que roda sobre o TensorFlow, CNTK, ou Theano.

Documentação: https://keras.io/

Versão: Maio, 2021

## O Problema

Classificar dados de sinais coletados da ionosfera terrestre.

Entrada: instâncias compostas por 33 atributos representando os sinais coletados.

Saída: "G" (Good): o radar retorna que os elétrons livres na ionosfera mostram a evidência de algum tipo de estrutura; "B" (Bad): os elétrons livres não indicam estrutura.

## Arquitetura Inicial

![neuronio-sigmoide-ajustado.png](attachment:neuronio-sigmoide-ajustado.png)

<p>
Será implementado um classificador binário usando a regressão logística (Logistic Regression Classifier) para resolver o problema.
    
**Expressões Matemáticas**:

Para um exemplo $x^{(i)}$:
$$z^{(i)} = w^T x^{(i)} + b$$
$$\hat{y}^{(i)} = a^{(i)} = sigmoid(z^{(i)})$$ 
$$ \mathcal{L}(a^{(i)}, y^{(i)}) =  - y^{(i)}  \log(a^{(i)}) - (1-y^{(i)} )  \log(1-a^{(i)})$$

O custo é computado somando-se todos os exemplos de treino:
$$ J = \frac{1}{m} \sum_{i=1}^m \mathcal{L}(a^{(i)}, y^{(i)})$$

## Pacotes

In [3]:
import numpy as np # package for scientific computing
import tensorflow as tf  #  package for numerical computation using data flow graphs
from tensorflow import keras  # package for deep learning
import pandas as pd # package for working with structured data

from google.colab import files
uploaded = files.upload()



Saving ionosphere_data.csv to ionosphere_data.csv


## Dados para Treino e Teste

Fonte: https://www.kaggle.com/prashant111/ionosphere?select=ionosphere_data.csv

In [4]:
df = pd.read_csv('ionosphere_data.csv')
df['column_a'] = df.column_a.astype('float64') # convert boolean feature into numeric
df.drop(columns=['column_b'], inplace=True) # remove useless feature
df.head() # display dataset first lines


,column_a,column_c,column_d,column_e,column_f,column_g,column_h,column_i,column_j,column_k,...,column_z,column_aa,column_ab,column_ac,column_ad,column_ae,column_af,column_ag,column_ah,column_ai
0,1.0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,0.85243,...,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,g
1,1.0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,0.50874,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,b
2,1.0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,0.73082,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,g
3,1.0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,b
4,1.0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,0.52798,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,g


In [5]:
# Encoding the output class
df.rename(columns={'column_ai': 'label'}, inplace=True)
df['label'] = df.label.astype('category')
encoding = {'g': 1, 'b': 0}
df.label.replace(encoding, inplace=True)
df.head()

,column_a,column_c,column_d,column_e,column_f,column_g,column_h,column_i,column_j,column_k,...,column_z,column_aa,column_ab,column_ac,column_ad,column_ae,column_af,column_ag,column_ah,label
0,1.0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,0.85243,...,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,1
1,1.0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,0.50874,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,0
2,1.0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,0.73082,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,1
3,1.0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,0
4,1.0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,0.52798,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,1


Documentação de *train_test_split*: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html <br>
A função divide os dados em partições de treino e teste, de acordo com a proporção especificada pelo parâmetro *test_size*. <br>
O parâmetro *random_state* é usado para deixar os resultados reproduzíveis para fins de avaliação do exercício.

In [6]:
# Preparing the dataset for training and test
from sklearn.model_selection import train_test_split
X = df.values[:,:-1]
Y = df.values[:,-1]
train_set_X, test_set_X, train_set_Y, test_set_Y = train_test_split(X, Y, test_size=0.20, random_state=7)

Documentação de *fit_transform*: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html <br>
A função modifica o valor da cada atributo de cada instância dos dados com base na média e no desvio padrão do conjunto de dados.

In [7]:
# Standardize features by removing the mean and scaling to unit variance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# The boolean feature does not need to be normalized.
train_set_X[:,1:] = scaler.fit_transform(train_set_X[:,1:])
test_set_X[:,1:]  = scaler.fit_transform(test_set_X[:,1:])

### Atenção
No formato da entrada de dados da API Keras, as linhas representam as instâncias (exemplos) e as colunas representam os atributos. Isso é invertido em relação ao que foi visto até agora na implementação mostrada nos exemplos anteriores.

In [8]:
n = train_set_X.shape[1] # number of attributes
m = train_set_X.shape[0] # number of training examples

print ("Number of attributes: n = " + str(n))
print ("Number of training examples: m = " + str(m))
print ("Train set X shape: " + str(train_set_X.shape))
print ("Train set Y shape: " + str(train_set_Y.shape))
print ("Test set X shape: " + str(test_set_X.shape))
print ("Test set Y shape: " + str(test_set_Y.shape))

Number of attributes: n = 33
Number of training examples: m = 280
Train set X shape: (280, 33)
Train set Y shape: (280,)
Test set X shape: (71, 33)
Test set Y shape: (71,)


## Definição do Modelo

O modelo em Keras é definido como uma sequência de camadas.

No exemplo abaixo, a rede é constituida de apenas um neurônio, o qual recebe os dados da camada de entrada e produz a saída. Como é um problema de classificação binária, a saída é produzida pela função de ativação Sigmóide.

Documentações: <br>
Classe *Model*: https://keras.io/api/models/model/ <br>
Funções de ativação disponíveis: https://keras.io/api/layers/activations/ 

In [9]:
inputs = keras.Input(shape=(train_set_X.shape[1]))  # the shape of inputs (number of attributes)
outputs = keras.layers.Dense(units=1, activation="sigmoid")(inputs)  # single neuron network with sigmoid as activation function
model = keras.Model(inputs=inputs, outputs=outputs)  # instantiate a Model object

In [10]:
# Checking
processed_data = model(train_set_X)
print(processed_data.shape)

(280, 1)


In [11]:
# Prints a summary of the network, showing its architecture and parameters.
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 33)]              0         
                                                                 
 dense (Dense)               (None, 1)                 34        
                                                                 
Total params: 34
Trainable params: 34
Non-trainable params: 0
_________________________________________________________________


## Compilação do Modelo

Na compilação do modelo, a biblioteca de backend (TensorFlow, por exemplo) escolhe a melhor forma de representar a rede no seu hardware, tal como CPU, GPU ou TPU.

Os seguintes parâmetros devem ser especificados:
1. Loss Function – função de perda. No exemplo, será utilizada a função Cross-Entropy, usada na Regressão Logística.
2. Optimizer – otimizador, o qual executa o algoritmo da descida do gradiente. Um otimizador popular é chamado de "adam".
3. Metrics – métricas a serem monitoradas (opcional)

Documentação: https://keras.io/api/models/model_training_apis/

In [12]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

## Treinamento do Modelo

Ajusta o modelo aos dados de treinamento.
Devem ser fornecidos os dados de treinamento, o número de épocas (iterações) e o tamanho do lote (batch). Uma época é composta por uma única passagem por todos os exemplos do conjunto de treino. O tamanho do lote define o número de amostras (exemplos) a serem consideradas pelo modelo antes de atualizar os pesos. Assim, uma época é composta por um ou mais lotes.

Documentação: https://keras.io/api/models/model_training_apis/

In [13]:
history = model.fit(train_set_X, train_set_Y, batch_size=32, epochs=100)  # history records what happened over the course of training
print(history.history)  # print per-epoch timeseries of metrics values

Epoch 1/100
9/9 [==============================] - 1s 2ms/step - loss: 1.1350 - accuracy: 0.3321
Epoch 2/100
9/9 [==============================] - 0s 2ms/step - loss: 1.0891 - accuracy: 0.3429
Epoch 3/100
9/9 [==============================] - 0s 2ms/step - loss: 1.0444 - accuracy: 0.3500
Epoch 4/100
9/9 [==============================] - 0s 2ms/step - loss: 1.0023 - accuracy: 0.3571
Epoch 5/100
9/9 [==============================] - 0s 2ms/step - loss: 0.9613 - accuracy: 0.3607
Epoch 6/100
9/9 [==============================] - 0s 2ms/step - loss: 0.9240 - accuracy: 0.3643
Epoch 7/100
9/9 [==============================] - 0s 2ms/step - loss: 0.8891 - accuracy: 0.3786
Epoch 8/100
9/9 [==============================] - 0s 2ms/step - loss: 0.8566 - accuracy: 0.3857
Epoch 9/100
9/9 [==============================] - 0s 2ms/step - loss: 0.8262 - accuracy: 0.4036
Epoch 10/100
9/9 [==============================] - 0s 2ms/step - loss: 0.7967 - accuracy: 0.4179
Epoch 11/100
9/9 [===========

## Avaliação do Modelo

Avalia o desempenho da rede no conjunto de teste.

Documentação: https://keras.io/api/models/model_training_apis/

In [14]:
loss, acc = model.evaluate(test_set_X, test_set_Y)
print("loss: %.2f" % loss)
print("acc: %.2f" % acc)

3/3 [==============================] - 0s 5ms/step - loss: 0.4779 - accuracy: 0.8592
loss: 0.48
acc: 0.86


## Predição

Prediz a saída de novos dados.

Documentação: https://keras.io/api/models/model_training_apis/

In [15]:
predictions = model.predict(test_set_X)
print("Predictions: ", [round(x[0]) for x in predictions])
print("\nCorrect:     ", [round(x) for x in test_set_Y])

3/3 [==============================] - 0s 3ms/step
Predictions:  [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0]

Correct:      [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1]


# Fim

Esses foram os passos para criação de uma rede neural simples, composta de um único neurônio.

Com o incremento do número de camadas, a rede é capaz de aprender atributos mais complexos.
A seguir, vamos criar uma rede neural com mais camadas e aplicá-la para resolver o mesmo problema de classificação anterior.

-------------------------------------------

# Nova Arquitetura

Vamos criar agora um novo modelo, seguindo a arquitetura abaixo:

![arquitetura-rede-neural.png](attachment:arquitetura-rede-neural.png)

**Medindo o tamanho de uma rede:**
As duas métricas mais comumente usadas são o número de neurônios e o número de parâmetros (mais comum). A rede acima, considerando 33 atributos de entrada, tem:
- 3 + 4 + 1 = 8 neurônios (não contando as entradas) e
- (33 x 3) + (3 x 4) + (4 x 1) = 115 pesos e 3 + 4 + 1 = 8 biases, totalizando 123 parâmetros a serem aprendidos.

## Definição do Modelo em Keras

In [16]:
inputs = keras.Input(shape=(train_set_X.shape[1]))  # the shape of inputs (number of attributes)
x = keras.layers.Dense(units=3, activation="relu")(inputs)  ## first layer, 3 neurons, activation function ReLu
x = keras.layers.Dense(units=4, activation="relu")(x)  ## second layer, 4 neurons, activation function ReLu
outputs = keras.layers.Dense(units=1, activation="sigmoid")(x)  # output layer, 1 neuron, activation function Sigmoid
model = keras.Model(inputs=inputs, outputs=outputs)  # instantiate a Model object

In [17]:
# Prints a summary of the network, showing its architecture and parameters.
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 33)]              0         
                                                                 
 dense_1 (Dense)             (None, 3)                 102       
                                                                 
 dense_2 (Dense)             (None, 4)                 16        
                                                                 
 dense_3 (Dense)             (None, 1)                 5         
                                                                 
Total params: 123
Trainable params: 123
Non-trainable params: 0
_________________________________________________________________


## Compilação do Modelo

In [18]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

## Treinamento do Modelo

In [19]:
history = model.fit(train_set_X, train_set_Y, batch_size=32, epochs=100)  # history records what happened over the course of training
print(history.history)  # print per-epoch timeseries of metrics values

Epoch 1/100
9/9 [==============================] - 1s 2ms/step - loss: 0.6981 - accuracy: 0.4536
Epoch 2/100
9/9 [==============================] - 0s 2ms/step - loss: 0.6837 - accuracy: 0.4821
Epoch 3/100
9/9 [==============================] - 0s 2ms/step - loss: 0.6716 - accuracy: 0.5071
Epoch 4/100
9/9 [==============================] - 0s 3ms/step - loss: 0.6620 - accuracy: 0.5536
Epoch 5/100
9/9 [==============================] - 0s 2ms/step - loss: 0.6524 - accuracy: 0.7000
Epoch 6/100
9/9 [==============================] - 0s 2ms/step - loss: 0.6441 - accuracy: 0.7500
Epoch 7/100
9/9 [==============================] - 0s 2ms/step - loss: 0.6374 - accuracy: 0.7607
Epoch 8/100
9/9 [==============================] - 0s 2ms/step - loss: 0.6303 - accuracy: 0.7857
Epoch 9/100
9/9 [==============================] - 0s 4ms/step - loss: 0.6236 - accuracy: 0.8000
Epoch 10/100
9/9 [==============================] - 0s 2ms/step - loss: 0.6156 - accuracy: 0.8036
Epoch 11/100
9/9 [===========

## Avaliação do Modelo

In [23]:
loss, acc = model.evaluate(train_set_X, train_set_Y)
print("loss: %.2f" % loss)
print("acc: %.2f" % acc)

9/9 [==============================] - 0s 2ms/step - loss: 0.1719 - accuracy: 0.9464
loss: 0.17
acc: 0.95


## Predição

In [24]:
predictions = model.predict(train_set_X)
print("Predictions: ", [round(x[0]) for x in predictions])
print("\nCorrect:     ", [round(x) for x in train_set_Y])

9/9 [==============================] - 0s 2ms/step
Predictions:  [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1]

Correct:      [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 